In [ ]:
execfile('d:/measuring/analysis/scripts/setup_analysis.py')
%matplotlib inline


In [ ]:
nr_of_scans = 16
older_than=None
fig,ax=plt.subplots(1, figsize=(16,5))
for i in range(nr_of_scans):
    timestamp,folder = tb.latest_data(contains='LaserScan', older_than = older_than,return_timestamp=True)
    older_than=timestamp

    #print folder

    a=m2.M2Analysis(folder=folder)
    for k in a.f.keys():
        if '_data' in k:
            a.dg = a.f[k]
    a.counts,a.frequency,a.voltage = a.dg['Counts [Hz]'].value,a.dg['Frequency [GHz]'].value,a.dg['Voltage [V]'].value
    #print len(a.counts)
    if len(a.counts)>200:
        ax.plot(a.frequency, a.counts, label=' '+a.name)
        
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Countrate [Hz]')
#ax.legend()

In [ ]:
fig,ax=plt.subplots(1)


In [ ]:
tb.get_msmt_fp?

In [ ]:
older_than=None
newer_than = '20160920152010'
newer_than_old = '20160908080000'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
bins = np.arange(-500,500,.5)
hist = np.zeros(len(bins)-1)
bin_norm = np.zeros(len(bins)-1)
while 1:
    old_laser_scan=False
    ret = tb.latest_data(contains='LasefdsrScan', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        ret = tb.latest_data(contains='laser_scan', older_than = older_than,newer_than=newer_than_old,return_timestamp=True, raise_exc=False)
        old_laser_scan = True
        if not ret:
            break
    timestamp,folder = ret
    older_than=timestamp
    if old_laser_scan:
        d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
        if len(d)>2:
            voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    else:
        a=m2.M2Analysis(folder=folder)
        data_found = False
        for k in a.f.keys():
            if '_data' in k and 'area_5' in k:
                a.dg = a.f[k]
                data_found=True
        if not data_found:
            continue
        voltage, frequency, counts = a.dg['Voltage [V]'].value, a.dg['Frequency [GHz]'].value, a.dg['Counts [Hz]'].value
     
    if len(counts)>200:
        cts_norm_zero = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=cts_norm_zero, bins=bins)
        bin_norm+=1.0*(_h>0)
        hist+= _h
        ax2.plot(frequency,cts_norm_zero+1, alpha=0.5)
        
ax.plot(bins[1:],hist, '.')        
ax.set_xlim(200,260)
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Deviation from average countrates')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
ax2.set_xlim(200,260)
#ax.legend()

In [ ]:
older_than=None#'20160927204809'
newer_than = '20160928132457'
fig,ax=plt.subplots(1, figsize=(16,5))
fig2,ax2=plt.subplots(1, figsize=(16,5))
bins = np.arange(100,120,.05)
hist = np.zeros(len(bins)-1)
bin_norm = np.zeros(len(bins)-1)
ii = 0
while 1:
    ret = tb.latest_data(contains='laser_scan', older_than = older_than,newer_than=newer_than,return_timestamp=True, raise_exc=False)
    if not ret:
        break
    timestamp,folder = ret
    older_than=timestamp
    if not 'g_0.0' in folder:
        print folder
        continue        
    #print folder
    d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
    if len(d)>2:
        voltage, frequency, counts = d[:,0],d[:,1],d[:,2]
    if len(counts)>100:
        cts = counts.astype(np.float)/np.average(counts) -1
        _h,_b = np.histogram(frequency, weights=cts, bins=bins)
        hist+= _h
        ax2.plot(frequency,cts, alpha=0.5)
        ii+=1
        
ax.plot(bins[1:],hist)        
ax.set_xlim(min(bins),max(bins))
ax.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Deviation from average countrates')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax2.set_ylabel('Normalized countrates')
ax2.set_xlim(min(bins),max(bins))
#ax.legend()
print ii

In [ ]:
t = np.array([637.009, 637.04,637.06,637.153,637.289])
d = np.array([250, 232.1, 217.1, 153.4,49.4])

x,y=1/t,d
fig,ax=plt.subplots(1, figsize=(16,5))

f = common.fit_line
args=[0,100]
fitres = fit.fit1d(x, y, f,*args, fixed = [],
                   do_print = True, ret = True, maxfev=100)
p1 = fitres['params_dict']
ax.plot(x,y, 'o')
plot_pts=100
x_p=np.linspace(min(x),max(x),plot_pts)
if fitres['success']:
    f_p = fitres['fitfunc'](x_p)
    ax.plot(x_p,f_p)
f_g = f(*args)[1](x_p)
ax.plot(x_p,f_g)

In [ ]:
common.fit_line?

In [ ]:
print fitres['fitfunc'](1/637.16)
print fitres['fitfunc'](1/636.526)

In [ ]:
folder = r'd:\\measuring\\data\\20160921\\161331_laser_scan_Sophie_area_5_NV1_g_2.0_r_200.0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    frequency, counts,counts_no_green = d[:,0],d[:,1],d[:,2]

In [ ]:
fig,(ax,ax2)=plt.subplots(2, figsize=(16,8))
ax.plot(frequency,counts, label = 'Green + Red')
ax.plot(frequency, counts_no_green, label = 'Red only')
ax2.plot(frequency,counts-counts_no_green,'r',
         label = 'Difference')
ax2.set_xlabel('Frequency (GHz wrt 470.4 THz)')
ax.set_ylabel('Countrates [Hz]')
ax2.set_ylabel('Countrate difference [Hz]')
ax.legend()
ax2.legend()
#from analysis.lib.tools import rebin
#order = 1
#ax.plot(rebin.average(frequency,order=order),rebin.rebin(counts-counts_no_green,order=order))

In [ ]:
bins = np.arange(200,1050,.5)

hist,_b = np.histogram(frequency, weights=counts-counts_no_green, bins=bins)       
fig,ax2=plt.subplots(1, figsize=(16,5))
ax2.plot(bins[1:],hist, '.')   

In [ ]:
nvlevels.get_transitions_ExEy(75,93.8,
                              show_ms0_transitions=True,
                              show_m1_transitions=True,
                              show_p1_transitions=False,
                              show_A_transitions=False,
                              show_FB_A_transitions=False,
                              show_FB_E_transitions=False,
                              show_E_prime_flip_transitions=False)

In [ ]:
### Fitting laserscans 1 peak at a time

f_min = 70
f_max = 90
g_a= 200
g_A= 800
g_x0 = 80
g_gamma = 1
fixed=[]


folder = r'K:\ns\qt\Diamond\Samples\Cavity samples\2016-09 Harry_unetched\Laserscans\Scan5 NV1\090101_laser_scan_Harry_Scan5_NV1_g_1.0_r_30.0'
d = np.loadtxt(tb.get_msmt_fp(folder, ext='dat'), skiprows=15)
if len(d)>2:
    voltage, frequency, counts = d[:,0],d[:,1],d[:,2]

fig,ax=plt.subplots(1, figsize=(16,5))
plt.plot(frequency,counts,'-')
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
plt.savefig(os.path.join(folder,'Scan5_NV1_wholefigure.png'))

idx_min = np.searchsorted(frequency, f_min, side="left")
idx_max = np.searchsorted(frequency, f_max, side="left")
frequency = frequency[idx_min:idx_max]
counts = counts[idx_min:idx_max]

fig,ax=plt.subplots(1, figsize=(10,5))
    
p0, fitfunc, fitfunc_str = common.fit_lorentz(g_a, g_A, g_x0, g_gamma)
fit_result = fit.fit1d(frequency,counts, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, fixed=fixed)

T = fit_result['params_dict']['x0']
gamma = fit_result['params_dict']['gamma']
u_gamma = fit_result['error_dict']['gamma']

print 'T = ', T , 'GHz'
print 'Linewidth T', gamma, 'GHz +/-', u_gamma, 'GHz'


plot.plot_fit1d(fit_result, np.linspace(frequency[0],frequency[-1],len(frequency)),ax=ax, label='Fit',show_guess=True, plot_data=True,color='red', data_linestyle = '-', print_info= False)
ax.set_xlabel("Frequency (GHz wrt 470.4 THz)", fontsize = 14)
ax.set_ylabel("Countrate (Hz)", fontsize = 14)
ax.legend(title='Transition at %.2f GHz, Linewidth = %.2f $\pm$ %.2f MHz'%(T, gamma*1.e3,u_gamma*1.e3))
plt.savefig(os.path.join(folder,'Scan5_NV1_Ex.png'))






In [ ]:
print 91.18 - 79.23